---

<div style="display: flex; align-items: center; justify-content: space-between;">
  <img src="../../.images/PNAV-logo.png" alt="Logo del PNAV" style="width: auto; max-height: 100px;">
  <img src="../../.images/MITECO-logo_background.png" alt="Logo del MITECO" style="width: auto; max-height: 100px;">
</div>

---

# Inferencia con LLMs utilizando Transformers
A continuación, se mostrará el proceso de cómo hacer inferencia con el LLM `Qwen2.5-1.5B-Instruct` utilizando la librería `transformers` desde un notebook en Azure.

⚠️ Este material es un notebook de demostración **orientativo** sobre la metodología que se debe seguir, no se debe replicar el desarrollo del caso de uso.

## 1. Instalación de librerías y paquetes

In [1]:
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install transformers
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install torch
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install 'accelerate>=0.26.0'

The system cannot find the path specified.
The system cannot find the path specified.
The system cannot find the path specified.


## 2. Importación de módulos
Se importan las clases `AutoModelForCausalLM` y `AutoTokenizer` del paquete `transformers`. Estas clases permiten cargar modelos y tokenizadores preentrenados para tareas de modelado de lenguaje causal.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

## 3. Nombre del modelo
Se define el nombre del modelo preentrenado que se desea utilizar, en este caso `"Qwen/Qwen2.5-1.5B-Instruct"`, un modelo de lenguaje desarrollado por Alibaba Cloud.

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

## 4. Carga del modelo
- `AutoModelForCausalLM`:from_pretrained: Carga el modelo preentrenado desde el nombre especificado.
- `torch_dtype="auto"`: Configura automáticamente el tipo de dato (por ejemplo, `float32` o `float16`) en función del dispositivo disponible.
- `device_map="auto"`: Distribuye el modelo automáticamente en los dispositivos disponibles, como CPU o GPU.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

## 5. Carga del tokenizador
Carga el tokenizador correspondiente al modelo preentrenado. El tokenizador convierte el texto en tensores numéricos que pueden ser procesados por el modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 6. Definición del prompt
Se especifica el texto inicial que se proporcionará al modelo como entrada.

In [ ]:
prompt = "Give me a short introduction to large language model."

## 7. Definición del contexto
Se crea una lista de mensajes, con un mensaje inicial de sistema que define el rol del modelo y un mensaje de usuario que contiene el `prompt`.

In [ ]:
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

## 8. Preparación del texto
- `apply_chat_template`: Convierte la lista de mensajes en un formato de texto comprensible para el modelo.
- `tokenize=False`: Indica que el texto generado no debe tokenizarse todavía.
- `add_generation_prompt=True`: Añade automáticamente información necesaria para la generación de texto.

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

## 9. Tokenización y preparación para el modelo
- `tokenizer([text], return_tensors="pt")`: Convierte el texto en tensores de PyTorch (pt).
- `.to(model.device)`: Mueve los tensores al mismo dispositivo donde está el modelo (por ejemplo, GPU).

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

## 10. Generación de texto
- `model.generate`: El modelo genera texto a partir de los tensores de entrada.
- `max_new_tokens=512`: Especifica el número máximo de tokens que el modelo puede generar.

In [ ]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

## 11. Extracción de tokens generados
Filtra los tokens generados eliminando los tokens de entrada para quedarse solo con los nuevos tokens producidos por el modelo.

In [ ]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

## 12. Decodificación del texto generado
- `tokenizer.batch_decode`: Convierte los tokens numéricos de vuelta a texto legible.
- `skip_special_tokens=True`: Elimina tokens especiales del texto final.
- `[0]`: Selecciona el primer elemento (ya que normalmente se genera solo una respuesta).

In [ ]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

## 13. Impresión de la respuesta
Muestra el texto generado por el modelo como salida final.

In [11]:
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/anaconda/envs/azureml_py310

A large language model (LLM) is an artificial intelligence system that has the ability to generate human-like text based on vast amounts of data it has been trained on. These models use deep learning techniques and neural networks to understand and process natural language, allowing them to perform tasks such as translation, summarization, question-answering, and creative writing.

Some key features of LLMs include:

1. **Training Data**: They are typically trained using massive datasets, often in the billions or trillions of words, which allows them to learn complex patterns and relationships within natural language.

2. **Generative Abilities**: LLMs can produce novel text that resembles real human language, making them useful for applications like chatbots, content generation, and even poetry creation.

3. **Scalability**: Due to their extensive training, these models can handle very large inputs and outputs efficiently, making them suitable for processing long documents and generat

## 14. Disclaimer
⚠️ La forma de hacer inferencia de cada modelo puede variar. Para consultar cómo inferir con un determinado modelo, consultar la tarjeta del modelo en [Hugging Face](https://huggingface.co/).